In [ ]:
import os
import warnings
import sys


import gymnasium as gym
import numpy as np
import torch
from design import swimmer_design



from design import *
from utils import Train, Eva
import shutil
import importlib
from gymnasium.envs.robodesign.GPTSwimmer import GPTSwimmerEnv


import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTSwimmerEnv._get_rew = _get_rew



# Ensure parameters are within bounds
design_params=np.array([0.009999999776482582,0.009999999776482582,0.5,0.009999999776482582,0.009999999776482582,0.009999999776482582])
init_parameter = np.array([1, 1, 1, 0.1, 0.1, 0.1])
design_params = init_parameter + design_params

# Generate XML using ant_design function
xml_string = swimmer_design(design_params)

# Save to a temporary file first
tmp_xml = f"swimmer_design.xml"
with open(tmp_xml, "w") as f:
    f.write(xml_string)

# Copy to standard name that Train/Eva expect

shutil.copy(tmp_xml, "GPTSwimmer.xml")

# Create folder for training results
folder_name = f"results"
os.makedirs(folder_name, exist_ok=True)

print(f"Running Train() with {design_params}")

morphology_index=222
rewardfunc_index=222

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)

fitness, _ = Eva(model_path)
material = compute_swimmer_volume(design_params)
efficiency = fitness / material

print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

with open('eff_fit.txt', 'w') as f:

        f.write(f"fitness:{fitness}")
        f.write(f"efficiency:{efficiency}")

        f.write("\n")

In [ ]:
fitness:8.660553278102386
efficiency:57.53276025499397